In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from dg_functions_base_matlab import *

In [3]:
%reload_ext autoreload
%autoreload 2

In [ ]:

#Input Data
#nelem=3; %Number of Elements == 22
#nop1=2;
#nop2=2;    %Interpolation Order == 5

nop_array = [2,4,6,8]
nelem_array = [20,40,80,160]

kstages = 4 #RK2, RK3, RK34
dt = 1e-2  #time-step, fraction of one revolution
Courant_max = 0.05
time_final = 0.025  #final time in revolutions
nplots = 50 #plotting variable - Ignore
iplot_solution=0 #Switch to Plot or Not.
interpolation_points=1  # %=1 for LGL; =2 equi-spaced points; =3 for LG
integration_points=1 # %=1 for LGL and =2 for LG
integration_type=2 # %=1 is inexact and =2 is exact
space_method_type='dg'# %=1 for CG and =2 for DG

icase=1  # case number: 1 is a Gaussian, 2 is a square wave, 3 is
         #Gaussian with source and 4 is square wave with source.
         
#stabilization - won't need it for now         
xmu = 0.0 # filtering strength: 1 is full strength and 0 is no filter
ifilter=10000  #time-step frequency that the filter is applied.
diss = 0 
#visc=0.000 #for CG N=4 and N=5
nlaplacian=0

#if (nlaplacian == 0) 
#    visc=0;
#end

#Store Constants
# ngl=nop + 1;
ntime = time_final/dt;
l2_norm = zeros((len(nop_array), len(nelem_array)))
npoin_array = zeros((len(nop_array), len(nelem_array)))

for i_nop in range(len(nop_array)):
    nop = nop_array[i_nop]
    ngl = nop+1
    
    for i_nelem in range(len(nelem_array)):
        nelem = nelem_array[i_nelem]
        method_text = [space_method_type]
        if (space_method_type == 'cg'):
            npoin = ngl*nelem
        elif (space_method_type == 'dg'):
            npoin = ngl*nelem
        
        npoin_array[i_nop,i_nelem] = npoin
        #Compute Interpolation and Integration Points
        if (interpolation_points == 1):
            [xgl,wgl] = legendre_gauss_lobatto(ngl)
            method_text = [method_text, '_Interp=LGL']
        elif (interpolation_points == 2):
            wgl = 1
            xgl = linspace(-1,1,ngl)
            method_text=[method_text, '_Interp=ESP']
        elif (interpolation_points == 3):
            [xgl,wgl]=legendre_gauss(ngl)
            method_text=[method_text, '_Interp=LG']
        
        #xgl,wgl=legendre_gauss_lobatto(ngl)
        #method_text='_Interp=LGL'
        
        if (integration_points == 1):
            integration_text=['LGL']
            if (integration_type == 1):
                noq=nop
            elif (integration_type == 2):
                noq=nop+1
            
            nq=noq+1
            [xnq,wnq] =legendre_gauss_lobatto(nq)
        elif (integration_points == 2):
            integration_text=['LG']
            noq=nop
            nq=noq + 1
            [xnq,wnq]=legendre_gauss(nq)
            
        method_text=[method_text, '_Integr=', integration_text]
        main_text=[method_text, ': ', integration_text]
        #Compute Lagrange Polynomial and derivatives
        psi,dpsi = lagrange_basis3(ngl,nq,xgl,xnq)

        #Compute Filter Matrix
        #f = filter_init(ngl,xgl,xmu)

        #Create Grid
        coord,intma = create_grid_dg(ngl,nelem,xgl)
        dx = coord[1,0]-coord[0,0]
        u=2
        dt=Courant_max*dx/u
        ntime = int(time_final/dt)
        dt=time_final/ntime
        Courant=u*dt/dx

        #Compute Exact Solution
        time=0
        qe = exact_solution_dg(coord,nelem,ngl,time,icase)
        #fe = source_function_dg(coord,nelem,ngl,time,icase)

        #Create Local/Element Mass and Differentiation Matrices
        mass = create_mass_dg(coord,nelem,ngl,nq,wnq,psi)
        
        diff_matrix = create_diff_matrix_dg(ngl,nq,wnq,psi,dpsi)
        laplacian_matrix = create_Lmatrix_dg(coord,nelem,ngl,nq,wnq,dpsi)

        #Form Global Matrix and Periodic BC Pointers
        inode=zeros((ngl,nelem))
        iperiodic=empty(npoin)
        
        if space_method_type == 'cg':
            inode=intma
            for i in range(npoin):
                iperiodic[i]=i
            iperiodic[npoin-1]=iperiodic[0]
        elif space_method_type == 'dg':
            
            ip=0
            for e in range(1,nelem+1):
                inode[0,e-1]=ip
                for i in range(1,ngl):
                    ip=ip+1
                    inode[i,e-1]=ip
                ip = ip+1

            for i in range(npoin):
                iperiodic[i] = i
    
        #Form Global Mass and Differentiation Matrices
        Mmatrix=zeros((npoin,npoin))
        Dmatrix=zeros((npoin,npoin))
        Fmatrix=zeros((npoin,npoin))
        Lmatrix=zeros((npoin,npoin))
        for e in range(1,nelem+1):
            for i in range(ngl):
                ip=iperiodic[int(inode[i,e-1])]
                ip = int(ip)
                for j in range(ngl):
                    jp=iperiodic[int(inode[j,e-1])]
                    jp = int(jp)
                    Mmatrix[ip,jp]=Mmatrix[ip,jp] + mass[e][i,j]
                    Dmatrix[ip,jp]=Dmatrix[ip,jp] + u*diff_matrix[i,j]
                    Lmatrix[ip,jp]=Lmatrix[ip,jp] + laplacian_matrix[e][i,j]
        
        if space_method_type == 'cg':
            Mmatrix[npoin-1,npoin-1]=1
            
        elif space_method_type == 'dg':
            Fmatrix = create_Fmatrix_dg(inode,npoin,nelem,ngl,u,diss)
            
        #Lmatrix_hat = solve(Mmatrix,Lmatrix)

        #Construct Full RHS matrix
        #if (nlaplacian > 0):
        #    HV_matrix = eye(npoin)
        #elif (nlaplacian == 0):
        #    HV_matrix = zeros(npoin)
        # 
        #for i in range(nlaplacian):
        #    HV_matrix = Lmatrix_hat*HV_matrix
        
        Rmatrix = solve(Mmatrix,(Dmatrix - Fmatrix))
        #Rmatrix = linalg.inv(Mmatrix)@(Dmatrix-Fmatrix)
        
        #Left-Multiply by Inverse Mass Matrix
        Dmatrix_hat = Rmatrix
        
        #Form Long Exact Solution Vector
        qexact=zeros(npoin)
        for e in range(1,nelem+1):
            for i in range(ngl):
                ip = int(inode[i,e-1])
                qexact[ip] = qe[i,e-1]

        #Initialize State Vector
        qexact = transpose(qexact)
        q1 = qexact
        q0 = qexact
        qp = qexact
        iplot = round(ntime/nplots)
        
        #Time Integration
        for itime in range(ntime):
            time = time + dt

            for ik in range(1,kstages+1):
                if (kstages == 2):
                    if(ik == 1):
                        a0 = 1
                        a1 = 0
                        beta = 1
                    elif(ik == 2):
                        a0 = 0.5
                        a1 = 0.5
                        beta = 0.5
                elif(kstages == 3):
                    if(ik == 1):
                        a0=1
                        a1=0
                        beta=1
                        
                    if(ik == 2):
                        a0=3/4
                        a1=1/4
                        beta=1/4 
                    if(ik == 3):
                        a0 = 1/3
                        a1 = 2/3
                        beta = 2/3
                    
                elif(kstages == 4):
                    if(ik == 1):
                        a0=1
                        a1=0
                        beta=1/2

                    if(ik == 2):
                        a0=0
                        a1=1
                        beta=1/2

                    if(ik == 3):
                        a0 = 2/3
                        a1 = 1/3
                        beta = 1/6

                    if (ik == 4):
                        a0 = 0
                        a1 = 1
                        beta = 1/2
                        
                dtt = dt*beta
                
                qp = a0*q0 + a1*q1 + dtt*Dmatrix_hat@qp
                
                #apply Periodic bc
                if (space_method_type == 'cg'):
                    ip = periodic[npoin]
                    qp[npoin]=qp[int(ip)] 
              
                #Update
                q1 = qp

            #Filter Solution
            #if (mod(itime,ifilter) == 0):
            #    rhs = apply_filter_dg(qp,f,nelem,ngl)
            #    qp=rhs

            #Update Q
            q0 = qp
        
        #print('q0 = ',q0)
        #Compute Exact Solution
        qe = exact_solution_dg(coord,nelem,ngl,time,icase)
        #print(qe)
        #Form Long Exact Solution Vector
        for e in range(1,nelem+1):
            for i in range(ngl):
                ip = int(inode[i,e-1])
                qexact[ip] = qe[i,e-1]
        #print(qexact)
        #Compute Norm
        top = 0
        bot = 0
        error = zeros((npoin, npoin))
        for i in range(npoin):
            top = top + (q0[i]-qexact[i])**2
            error[i] = q0[i]-qexact[i]
            bot = bot + (qexact[i])**2
        
        #print(top)
        l2_norm[i_nop,i_nelem] = sqrt(top/bot)

        #Compute a gridpoint solution
        x_sol = zeros(npoin);
        for ie in range(1,nelem+1):
            for i in range(ngl):
                ip = int(inode[i,ie-1])
                x_sol[ip] = coord[i,ie-1]
        
        if(iplot_solution == 1):
            figure(i_nop)
            plot(x_sol, qexact)
            plot(x_sol, q0, '--')
            

In [ ]:
import cg_graphics
figure(5)
clf()
P = []   
for i in range(len(nop_array)):
    N = nop_array[i]
    N = nop_array[i]
    loglog(npoin_array[i], l2_norm[i], '-*',markersize=5, label = 'N = {}'.format(N))
    p = polyfit(log(npoin_array[i]), log(l2_norm[i]), 1)
    P.append(p[0])
    
xlabel('DOF')
ylabel('Error (Max-error)')
title('Error vs number of DOF')
#cg_graphics.set_xticks(NE*N)  # Create nice tick marks for NE
grid(axis='both',linestyle='--')
legend()
show()   

In [ ]:
for i in range(len(nop_array)):
    print('N = {:d}, rate = {:f}'.format(nop_array[i],P[i]))